<a href="https://colab.research.google.com/github/matheusfcorreia/labs/blob/master/oito_rainhas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import random

In [0]:
def gerar_populacao(tamanho, num_cromossomos):
  return np.random.randint(low = 0, high = 7, size = (tamanho, num_cromossomos))

In [0]:
def calcular_fitness(populacao):
  pontos_fitness = np.zeros(32)
  for i in range(0, 32):
    fitness = 0
    cromossomo = populacao[i]

    #Para cada gene (rainha), verifica quais outros genes estão na mesma linha
    #ou em diagonais
    for gene in range(0, cromossomo.size):
      for gene_comparado in range(gene + 1, cromossomo.size):
        if cromossomo[gene_comparado] == cromossomo[gene]:
          fitness += 1
        elif cromossomo[gene_comparado] == (cromossomo[gene] + (gene_comparado - gene)):
          fitness += 1
        elif cromossomo[gene_comparado] == (cromossomo[gene] - (gene_comparado - gene)):
          fitness+= 1
      pontos_fitness[i] = fitness
  return pontos_fitness

In [0]:
def crossover(cromossomo_1, cromossomo_2):
  #Estabelece aleatoriamente um ponto de corte no qual os cromossomos pais serão
  #seccionados para gerar os filhos
  ponto_de_corte = random.randint(1,6)
  cromossomo_filho_1 = np.zeros(8)
  cromossomo_filho_2 = np.zeros(8)
  
  cromossomo_filho_1[0:ponto_de_corte] = cromossomo_1[0:ponto_de_corte]
  cromossomo_filho_2[0:ponto_de_corte] = cromossomo_2[0:ponto_de_corte]

  cromossomo_filho_1[ponto_de_corte:] = cromossomo_2[ponto_de_corte:]
  cromossomo_filho_2[ponto_de_corte:] = cromossomo_1[ponto_de_corte:]

  return (cromossomo_filho_1, cromossomo_filho_2)

In [0]:
def mutacao(cromossomo):
  #Define um ponto aleatório para sofrer a mutação
  indice_mutacao = random.randint(0, 7)

  #Garante que o valor mutado será diferente do anterir
  valor_antigo = cromossomo[indice_mutacao]
  while valor_antigo == cromossomo[indice_mutacao]:
    cromossomo[indice_mutacao] = random.randint(0,7)
  
  return cromossomo

In [0]:
def calcular_chances(pontos_fitness):
  total = np.sum(pontos_fitness)
  chances = np.zeros(pontos_fitness.size)
  
  #Atribui os pontos de formma inversamente proporcional à fitness quando comparada ao total,
  #já que quanto menor a fitness, melhor o desempenho
  for i in range(0, pontos_fitness.size):
    chances[i] = round(1/(pontos_fitness[i]/total), 2)
  return chances

In [0]:
def algoritmo_genetico():
  encontrado = False
  execucoes = 0
  while not encontrado:
    execucoes += 1
    populacao = gerar_populacao(32, 8)
    solucao_index = 0
    for i in range (0, 50):
      print('---------------------------------------------------------------------------------')
      print('Geração ' + str(i))
      print('---------------------------------------------------------------------------------')
      pontos_fitness = calcular_fitness(populacao)
      for k in range(0, 32):
        print('Geração ' + str(i) + ' - Fitness do cromossomo ' + str(populacao[k]) + ': ' + str(pontos_fitness[k]))
      
      #Verifica se algum dos cromossomos é objetivo
      objetivo = np.where(pontos_fitness == 0)
      if objetivo[0].size > 0:
        solucao_index = objetivo[0][0]
        print('---------------------------------------------------------------------------------')
        print('Solução encontrada!')
        encontrado = True
        break;

      #Calcula as chances de cada cromossomo de cair na roleta
      chances = calcular_chances(pontos_fitness)
      roleta = np.zeros(int(np.sum(chances)))
      roleta_index = 0 

      #Atribui chances[i] espaços na roleta para o cromossomo populacao[i]
      for i in range(0, chances.size):
        for k in range(roleta_index, roleta_index+int(chances[i])):
          roleta[k] = i
        roleta_index += int(chances[i])

      nova_geracao = np.copy(populacao)

      #Executa os operadores de crossover e mutação
      for i in range(0, 32, 2):
        #Pega os índices dos pais através da roleta
        primeiro_pai_index = int(roleta[random.randint(0, roleta.size-1)])
        segundo_pai_index = int(roleta[random.randint(0, roleta.size-1)])

        #Garante que os pais não são o mesmo cromossomo (apenas verifica os índices, valores podem ser iguais)
        while segundo_pai_index == primeiro_pai_index:
          segundo_pai_index = int(roleta[random.randint(0, roleta.size-1)])
        
        pai_1 = populacao[primeiro_pai_index]
        pai_2 = populacao[segundo_pai_index]
        filho_1, filho_2 = crossover(pai_1, pai_2)
        print('Crossover com cromossomos ' + str(pai_1) + ' com ' + str(pai_2))
        print('Resultado do crossover: ' + str(filho_1) + ' - ' + str(filho_2))

        #Atribui os filhos à nova geração
        nova_geracao[i] = filho_1
        nova_geracao[i+1] = filho_2
      
      #Executa o operador de mutação (10% de chance)
      for i in range(0, 32):
        chance_mutacao = random.randint(0, 9)
        if chance_mutacao == 1:
          print('Mutação no cromossomo ' + str(nova_geracao[i]))
          nova_geracao[i] = mutacao(nova_geracao[i])
          print('Resultado da mutação: ' + str(nova_geracao[i]))
      
      populacao = nova_geracao
    
    if not encontrado:
      print('---------------------------------------------------------------------------------')
      print('Solução não encontrada na execução ' + str(execucoes) + '. Gerando nova população inicial aleatória')
      print('---------------------------------------------------------------------------------')
  print('---------------------------------------------------------------------------------')
  print('Solução encontada em ' + str(execucoes) + ' execuções: ' + str(populacao[solucao_index]))

algoritmo_genetico()

Streaming output truncated to the last 5000 lines.
Geração 24 - Fitness do cromossomo [0 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness do cromossomo [0 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness do cromossomo [6 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness do cromossomo [0 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness do cromossomo [0 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness do cromossomo [0 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness do cromossomo [6 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness do cromossomo [0 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness do cromossomo [0 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness do cromossomo [0 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness do cromossomo [0 2 5 1 1 4 0 3]: 2.0
Geração 24 - Fitness do cromossomo [0 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness do cromossomo [6 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness do cromossomo [0 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness do cromossomo [0 2 5 1 1 4 0 3]: 2.0
Geração 24 - Fitness do cromossomo [0 2 5 1 1 4 6 3]: 2.0
Geração 24 - Fitness 